In [5]:
import pandas as pd

In [6]:
# Update AVFT input files

# Load original template data
county = "King"

In [7]:
def interpolate_rates(county, veh_type_list, initial_year, final_year):

       # Load the original template data
       df = pd.read_csv(f'Y:/Air Quality/2026_2050_RTP/wa_dept_of_ecology_data/2023_MOVES_inputs_revised/{county}/{county}_avft_2023.csv')

       # Loop through each vehicle type
       for veh_type in veh_type_list:
              
              filter = (
                     (df['fuelTypeID'] == 9) & 
                     (df['modelYearID']==initial_year) & 
                     (df['sourceTypeID']==veh_type)
              )
                     
              if len(df.loc[filter,'fuelEngFraction'].values) == 1:
                     # Get the initial value for fuelEngFraction
                     initial_value = df.loc[filter,'fuelEngFraction'].values[0]

              initial_value = df.loc[(df['fuelTypeID'] == 9) & 
                     (df['modelYearID']==initial_year) & 
                     (df['sourceTypeID']==veh_type),'fuelEngFraction'].values[0]
              final_value = 1 # Assert

              # Fill in intermediate years
              for year in range(initial_year, final_year+1):
                     # Calculate the scale factor for the current year
                     scale_factor = (year - initial_year) / (final_year - initial_year)

                     # Get the actual original value for fuelEngFraction (the "initial_value" field is only used for linear interpolation)
                     original_value = df.loc[(df['fuelTypeID'] == 9) & 
                            (df['modelYearID']==year) & 
                            (df['sourceTypeID']==veh_type),'fuelEngFraction'].values[0]

                     # Calculate the new value for fuelEngFraction for EVs
                     new_value = initial_value * (1 - scale_factor) + final_value * scale_factor
                     # Update the DataFrame for the current year and sourceTypeID 21
                     df.loc[(df['fuelTypeID'] == 9) & 
                            (df['modelYearID'] == int(year)) & 
                            (df['sourceTypeID']==veh_type),'fuelEngFraction'] = new_value
                     

                     # Reduce fuelEngFraction for other fuel types
                     other_filter = (
                            (df['fuelTypeID'] != 9) & 
                            (df['modelYearID'] == int(year)) & 
                            (df['sourceTypeID']==veh_type)
                     )
                     other_frac = df.loc[other_filter,'fuelEngFraction']
                     
                     new_vals = other_frac - (other_frac/other_frac.sum())*(new_value-original_value)
                     df.loc[other_filter,'fuelEngFraction'] = new_vals

              # Fill in remaining years with 1 for EV and 0 for other fuel types
              for year in range(final_year+1, df['modelYearID'].max()):
                     # Update the DataFrame for the current year and sourceTypeID 21
                     df.loc[(df['fuelTypeID'] == 9) & 
                            (df['modelYearID']>= int(final_year)) & 
                            (df['sourceTypeID']==veh_type),'fuelEngFraction'] = 1
                     
                     df.loc[(df['fuelTypeID'] != 9) & 
                            (df['modelYearID']>= int(final_year)) & 
                            (df['sourceTypeID']==veh_type),'fuelEngFraction'] = 0

       
       return df
# Scale down the other fuel types as well
# repeat for other source types

# Process heavy and medium trucks as well


In [ ]:
# Create a scenario where all vehicles are electric by 2040
# Scale fuelTypeID up to 1 and all other values down to 0
# Only apply to light-duty vehicles (sourceTypeID 21, 31, 32)
veh_type_list = [21,31,32]

# Process these one sourceTypeID at a time
initial_year = 2026
final_year = 2040

# df = interpolate_rates(county, veh_type_list, initial_year, final_year)

for county in ['King','Kitsap','Pierce','Snohomish']:
    df = interpolate_rates(county, veh_type_list, initial_year, final_year)
    df.to_csv(f'Y:/Air Quality/2026_2050_RTP/scenarios/2040_full_light_duty_EV/2023_MOVES_inputs_revised/{county}/{county}_avft_2023.csv', index=False)
